In [2]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException, ElementNotVisibleException
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from urllib.request import urlopen
from selenium.webdriver.common.keys import Keys
import urllib.request
from urllib.request import HTTPError
import os
import csv
import time
import codecs
from time import sleep

In [30]:
class gCrawller:
    count = 0 # 현재까지 크롤링된 URL 개수
    
    #크롤러 객체 초기화
    def __init__(self, chromedriver_path, keyword, url_csv_filepath, PAUSE_TIME = 2):
        self.keyword = keyword
        self.target_url = 'https://www.google.co.kr/search?q='+keyword+'&source=lnms&tbm=isch&sa=X&ved=0ahUKEwic-taB9IXVAhWDHpQKHXOjC14Q_AUIBigB&biw=1842&bih=990'
        self.PAUSE_TIME = PAUSE_TIME
        self.driver = webdriver.Chrome(chromedriver_path)
        self.url_csv_filepath = url_csv_filepath
        gCrawller.count = 0
       
    #크롤러 객체 준비
    def crawlling_ready(self):
        self.driver.implicitly_wait(3) 
        self.driver.get(self.target_url)
        
        time.sleep(self.PAUSE_TIME)
        self.first_img = self.driver.find_element_by_xpath('//*[@id="rg_s"]/div[1]')
        self.first_img.click()
        
        time.sleep(self.PAUSE_TIME)
        self.next_btn = self.driver.find_element_by_xpath('//a[@id="irc-rab"]')
        self.more_btn = self.driver.find_element_by_xpath('//*[@id="irc_cc"]/div[2]/div[3]/div[3]/div/div[2]/div[8]/a/span')
        self.more_btn.click()
        
    def back_and_start_over(self):
        url = self.driver.current_url
        item = []
        item.append(url)
        self.write_csv_url(item)
        gCrawller.count+=1
        self.driver.back()
        time.sleep(self.PAUSE_TIME)
        
        try:
            self.next_btn.click()
        
        except StaleElementReferenceException:
            time.sleep(self.PAUSE_TIME)
            self.next_btn = self.driver.find_element_by_xpath('//a[@id="irc-rab"]')
            self.next_btn.click()
            
        time.sleep(self.PAUSE_TIME)
        self.driver.refresh()
        time.sleep(self.PAUSE_TIME)
        
        while True:
            try:
                self.more_btn = self.driver.find_element_by_xpath('//*[@id="irc_cc"]/div[2]/div[3]/div[3]/div/div[2]/div[8]/a/span')
                self.more_btn.click()
                break
            
            except NoSuchElementException:
                self.next_btn = self.driver.find_element_by_xpath('//a[@id="irc-rab"]')
                time.sleep(self.PAUSE_TIME)
                self.next_btn.click()
                self.driver.refresh()
                #break
                
            except ElementNotVisibleException:
                time.sleep(self.PAUSE_TIME)
                self.more_btn = self.driver.find_element_by_xpath('//*[@id="irc_cc"]/div[2]/div[3]/div[3]/div/div[2]/div[8]/a/span')    
    
    def write_csv_url(self,list):
        with codecs.open(self.url_csv_filepath,"a","utf-8") as fp:
            writer=csv.writer(fp,delimiter=",")
            writer.writerow(list)
    
    def main(self):
        self.crawlling_ready()
        while True:
            if gCrawller.count>20:
                break
            self.back_and_start_over()
            time.sleep(self.PAUSE_TIME)

In [31]:
#keyword = input("검색할 이미지를 입력하세요 : ")
keyword_list = ['피자', '파전']

for keyword in keyword_list:
    chromedriver_path='C:/Users/acorn/Desktop/weather/chromedriver_win32/chromedriver'
    url_csv_filepath = 'C:/Users/acorn/weather_bigdata/original_url/url_'+keyword+'.csv'
    gc = gCrawller(chromedriver_path, keyword, url_csv_filepath)
    gc.main()